In [12]:
import pandas as pd
from apyori import apriori

In [13]:
# 1. Load the dataset
base = pd.read_csv('MercadoSim.csv', delimiter=';', encoding='utf-8', header=None)

# 2. Create transactions representing the absence of an item AND the presence of others
all_rules = []
unique_items = pd.unique(base.values.ravel('K'))
unique_items = unique_items[~pd.isnull(unique_items)]

for item in unique_items:
    # Create transactions where the item is absent
    absent_transactions = base[base.isnull().any(axis=1) & (base != item).all(axis=1)].values.tolist()
    absent_transactions = [[elem for elem in trans if isinstance(elem, str)] + [f"NOT_{item}"] for trans in absent_transactions]

    # Apply Apriori
    regras = apriori(absent_transactions, min_support=0.3, min_confidence=0.8)
    all_rules.extend(list(regras))

# Filter and print rules where the antecedent is the absence of an item
filtered_rules = [rule for rule in all_rules if any(["NOT_" in item for item in rule[0]]) and len(rule[0]) > 1]
for rule in filtered_rules:
    print(rule)

In [14]:
# Listas para armazenar informações sobre as regras
Antecedente = []
Consequente = []
suporte = []
confianca = []
lift = []

# Processando cada regra filtrada
for rule in filtered_rules:
    s = rule[1]
    result_rules = rule[2]
    for result_rule in result_rules:
        a = list(result_rule[0])
        b = list(result_rule[1])
        c = result_rule[2]
        l = result_rule[3]
        
        # Filtrando regras que contêm 'nan' e garantindo que os antecedentes/consequentes não estejam vazios
        if 'nan' in a or 'nan' in b: continue
        if len(a) == 0 or len(b) == 0: continue
        
        # Adicionando os detalhes à lista
        Antecedente.append(a)
        Consequente.append(b)
        suporte.append(s)
        confianca.append(c)
        lift.append(l)

# Criando o DataFrame final com os detalhes das regras
RegrasFinais = pd.DataFrame({
    'Antecedente': Antecedente,
    'Consequente': Consequente,
    'Suporte': suporte,
    'Confiança': confianca,
    'Lift': lift
})

RegrasFinais

,Antecedente,Consequente,Suporte,Confiança,Lift
